In [1]:
import datetime as dt
import oracledb
import path_str
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import calc_form as f  # import functions
# import user defined functions from package
from udf import get_cant_ach, get_price_med, get_sum, get_num_min
from sqlalchemy import create_engine
import conn_str as conn  # import connection string
import matplotlib.pyplot as plt

==========> 1. Calculation

=> 1.1 Get Data Set

In [ ]:
try:    
    user = conn.conn_str["user"]
    pwd = conn.conn_str["pwd"]
    cp = oracledb.ConnectParams()
    cp.parse_connect_string(conn.conn_str["str"])

    %load_ext sql
    conn_str = f'oracle+oracledb://{user}:{pwd}@{cp.host}:{cp.port}/?service_name={cp.service_name}'
    %sql $conn_str
    sql_stm = %sql select * from ee_nom_r_ds_ibd where an = 2023 order by data asc
    tbl_data_set = pd.DataFrame(sql_stm).astype({"cant_ach_contr":float, "cost_ach_contr":float, "cost_med_contr":float, "cant_cump_pzu":float, "cost_cump_pzu":float, "cant_vz_pzu":float, "venit_vzu_pzu":float, "cant_ach_pzu":float,
                                                "pret_pzu":float, "cant_cump_pi":float, "cost_cump_pi":float, "cant_vz_pi":float, "venit_vz_pi":float, "cant_ach_pi":float, "pret_pi":float, "cant_cump_ech":float, "cost_cump_ech":float,
                                                "cant_vz_ech":float, "venit_vz_ech":float, "cant_ach_ech":float, "cost_med_ech_def":float, "cost_med_ech_exc":float, "cant_total":float, "cant_prog":float, "abat_abs_prog":float})

    print("Finish get data set!")
except Exception as err:
    print("Error message =>" + str(err))


=> 1.2 KPI Calculation : 
1. Loss quantity purchased on PZU & PI market and sold on PE market 
2. Loss quantity sold on PZU & PI market and purchased on PE market
3. Profit quantity purchased on PZU & PI market and sold on PE market
4. Profit quantity sold on PZU & PI market and purchased on PE market
5. Inefficiently traded quantity
6. Efficiently traded quantity

In [4]:
tbl_data_set["cant_net_pzu_pi_poz"] = tbl_data_set.apply(
    lambda row: get_cant_ach(row["cant_ach_pzu"], row["cant_ach_pi"], True), axis=1
)
tbl_data_set["cant_net_pzu_pi_neg"] = tbl_data_set.apply(
    lambda row: get_cant_ach(row["cant_ach_pzu"], row["cant_ach_pi"], False), axis=1
)
tbl_data_set["cant_net_ech_poz"] = tbl_data_set.apply(
    lambda row: get_cant_ach(row["cant_ach_ech"], 0, True), axis=1
)
tbl_data_set["cant_net_ech_neg"] = tbl_data_set.apply(
    lambda row: get_cant_ach(row["cant_ach_ech"], 0, False), axis=1
)
tbl_data_set["pret_med_cump_pzu_pi"] = tbl_data_set.apply(
    lambda row: get_price_med(
        get_sum(row["cost_cump_pzu"], row["cost_cump_pi"]),
        get_sum(row["cant_cump_pzu"], row["cant_cump_pi"]),
    ),
    axis=1,
)
tbl_data_set["pret_med_vz_pzu_pi"] = tbl_data_set.apply(
    lambda row: get_price_med(
        get_sum(row["venit_vzu_pzu"], row["venit_vz_pi"]),
        get_sum(row["cant_vz_pzu"], row["cant_vz_pi"]),
    ),
    axis=1,
)

# # 1. Loss => quantity purchased on PZU & PI market and sold on PE market

tbl_data_set["min_cump_pzu_pi_vz_ech"] = tbl_data_set.apply(
    lambda row: get_num_min(
        row["cant_net_pzu_pi_poz"], abs(row["cant_net_ech_neg"])),
    axis=1,
)
tbl_data_set["loss_cump_pzu_pi_vz_ech"] = tbl_data_set.apply(
    lambda row: f.form_loss_cump_pzu_pi_vz_ech(
        row["cant_net_pzu_pi_poz"],
        row["cant_net_ech_neg"],
        row["cost_med_ech_exc"],
        row["pret_med_cump_pzu_pi"],
        row["min_cump_pzu_pi_vz_ech"],
    ),  # type: ignore
    axis=1,
)


# 2. Loss => quantity sold on PZU & PI market and purchased on PE market

tbl_data_set["min_vz_pzu_pi_cump_ech"] = tbl_data_set.apply(
    lambda row: get_num_min(
        row["cant_net_ech_poz"], -row["cant_net_pzu_pi_neg"]),
    axis=1,
)
tbl_data_set["loss_vz_pzu_pi_cump_ech"] = tbl_data_set.apply(
    lambda row: f.form_loss_vz_pzu_pi_cump_ech(
        row["cant_net_pzu_pi_neg"],
        row["cant_net_ech_poz"],
        row["pret_med_vz_pzu_pi"],
        row["cost_med_ech_def"],
        row["min_vz_pzu_pi_cump_ech"],
    ),  # type: ignore
    axis=1,
)
# # 3. Profit => quantity purchased on PZU & PI market and sold on PE market

tbl_data_set["profit_cump_pzu_pi_vz_ech"] = tbl_data_set.apply(
    lambda row: f.form_profit_cump_pzu_pi_vz_ech(
        row["cant_net_pzu_pi_poz"],
        row["cant_net_ech_neg"],
        row["cost_med_ech_exc"],
        row["pret_med_cump_pzu_pi"],
        row["min_cump_pzu_pi_vz_ech"],
    ),  # type: ignore
    axis=1,
)


# 4. Profit => quantity sold on PZU & PI market and purchased on PE market

tbl_data_set["profit_vz_pzu_pi_cump_ech"] = tbl_data_set.apply(
    lambda row: f.form_profit_vz_pzu_pi_cump_ech(
        row["cant_net_pzu_pi_neg"],
        row["cant_net_ech_poz"],
        row["pret_med_vz_pzu_pi"],
        row["cost_med_ech_def"],
        row["min_vz_pzu_pi_cump_ech"],
    ),  # type: ignore
    axis=1,
)

# 5. Inefficiently traded quantity

tbl_data_set["cant_tranz_inef"] = tbl_data_set.apply(
    lambda row: f.form_cant_tranz_inef(
        row["loss_cump_pzu_pi_vz_ech"],
        row["loss_vz_pzu_pi_cump_ech"],
        row["min_cump_pzu_pi_vz_ech"],
        row["min_vz_pzu_pi_cump_ech"],
    ),  # type: ignore
    axis=1,
)

# 5. Efficiently traded quantity

tbl_data_set["cant_tranz_ef"] = tbl_data_set.apply(
    lambda row: f.form_cant_tranz_ef(
        row["profit_cump_pzu_pi_vz_ech"],
        row["profit_vz_pzu_pi_cump_ech"],
        row["min_cump_pzu_pi_vz_ech"],
        row["min_vz_pzu_pi_cump_ech"],
    ),  # type: ignore
    axis=1,
)

tbl_data_set["mod_timp"] = dt.datetime.now()

In [ ]:
df_filter = tbl_data_set[tbl_data_set["luna"] == 1]
tbl = df_filter["abat_abs_prog"]

df = pd.DataFrame(tbl)

# create a histogram of the values in the dataframe
ax = df.hist(column="abat_abs_prog", bins=5)

# calculate the mean and standard deviation of the values
mean = df["abat_abs_prog"].mean()
std = df["abat_abs_prog"].std()

# add a vertical line for the mean and standard deviation
ax[0, 0].axvline(mean, color="red", linestyle="dashed", linewidth=2)
ax[0, 0].axvline(mean + std, color="green", linestyle="dashed", linewidth=2)
ax[0, 0].axvline(mean - std, color="green", linestyle="dashed", linewidth=2)


# show the plot
plt.show()

=> 1.3 Calc Total Effect Net

In [ ]:
print(
    "Total Effect Net is : "
    + str(
        (
            tbl_data_set["loss_cump_pzu_pi_vz_ech"]
            + tbl_data_set["loss_vz_pzu_pi_cump_ech"]
            + tbl_data_set["profit_cump_pzu_pi_vz_ech"]
            + tbl_data_set["profit_vz_pzu_pi_cump_ech"]
        ).sum()
        / 1000000
    )
    + " mil lei "
)

==========> 2. Report / Analysis

=> 2.1 Quantity deviations FC vs REAL

In [ ]:
tbl_ibd_statistic_m = tbl_data_set.groupby(["an", "luna"], as_index=False)[
    ["abat_abs_prog"]
].describe()
tbl_ibd_statistic_m

In [ ]:
tbl_ibd_statistic_ytd = tbl_data_set.groupby(["an"], as_index=False)[
    ["abat_abs_prog"]
].describe()
tbl_ibd_statistic_ytd

In [ ]:
graph_abat = px.line(
    tbl_data_set,
    x="data",
    y="abat_abs_prog",
    title="Abatere absoluta prognoza vs realizat (%)",
    labels=dict(abat_abs_prog="Abatere [%]", data="Perioada"),
)
graph_abat.show()

In [ ]:
graph_prog_vs_real = px.line(
    tbl_data_set,
    x="data",
    y=["cant_total", "cant_prog"],
    title="Evolutie prognoza vs realizat (MWh)",
    labels=dict(value="Cantitate (MWh)", data="Perioada", variable="Legenda"),
)
graph_prog_vs_real.show()

=> 2.2 Calc Total Net Effect

In [ ]:
tbl_rap_ov = tbl_data_set.groupby(["an", "luna"], as_index=False).agg(
    {
        "loss_cump_pzu_pi_vz_ech": sum,
        "loss_vz_pzu_pi_cump_ech": sum,
        "profit_cump_pzu_pi_vz_ech": sum,
        "profit_vz_pzu_pi_cump_ech": sum,
        "cant_tranz_inef": sum,
        "cant_tranz_ef": sum,
        "cant_cump_pzu": sum,
        "cant_vz_pzu": sum,
        "cant_cump_pi": sum,
        "cant_vz_pi": sum,
        "cant_cump_ech": sum,
        "cant_vz_ech": sum,
    }
)

tbl_rap_ov[
    "Total cantitate tranzactionata ( val.abs.) in PZU,PI si PE (GWh)"
] = tbl_rap_ov.apply(
    lambda row: get_sum(
        row["cant_cump_pzu"],
        abs(row["cant_vz_pzu"]),
        row["cant_cump_pi"],
        abs(row["cant_vz_pi"]),
        row["cant_cump_ech"],
        abs(row["cant_vz_ech"]),
    )
    / 1000,
    axis=1,
)
tbl_rap_ov["Cantitate tranzactionata ineficient [GWh]"] = (
    tbl_rap_ov["cant_tranz_inef"] / 1000 + tbl_rap_ov["cant_tranz_ef"] / 1000
)
tbl_rap_ov["Cantitate cu impact negativ [GWh]"] = tbl_rap_ov["cant_tranz_inef"] / 1000
tbl_rap_ov["Cantitate cu impact pozitiv [GWh]"] = tbl_rap_ov["cant_tranz_ef"] / 1000

tbl_rap_ov[
    "Cantitate tranzactionata ineficient raportat la cantitate totala tranzactionata in PZU, PI si PE [%]"
] = tbl_rap_ov.apply(
    lambda row: get_price_med(
        row["Cantitate tranzactionata ineficient [GWh]"],
        row["Total cantitate tranzactionata ( val.abs.) in PZU,PI si PE (GWh)"],
    ),
    axis=1,
)

tbl_rap_ov["Valoare impact - tranzactii ineficiente [mil lei]"] = tbl_rap_ov.apply(
    lambda row: get_sum(
        row["loss_cump_pzu_pi_vz_ech"],
        row["loss_vz_pzu_pi_cump_ech"],
        row["profit_cump_pzu_pi_vz_ech"],
        row["profit_vz_pzu_pi_cump_ech"],
    )
    / 1000000,
    axis=1,
)
tbl_rap_ov["Valoare cu impact negativ [mil lei]"] = tbl_rap_ov.apply(
    lambda row: get_sum(row["loss_cump_pzu_pi_vz_ech"],
                        row["loss_vz_pzu_pi_cump_ech"])
    / 1000000,
    axis=1,
)
tbl_rap_ov["Valoare cu impact pozitiv [mil lei]"] = tbl_rap_ov.apply(
    lambda row: get_sum(
        row["profit_cump_pzu_pi_vz_ech"], row["profit_vz_pzu_pi_cump_ech"]
    )
    / 1000000,
    axis=1,
)


# housekeeping
tbl_rap_ov.drop(
    [
        "loss_cump_pzu_pi_vz_ech",
        "loss_vz_pzu_pi_cump_ech",
        "profit_cump_pzu_pi_vz_ech",
        "profit_vz_pzu_pi_cump_ech",
        "cant_tranz_inef",
        "cant_tranz_ef",
        "cant_cump_pzu",
        "cant_vz_pzu",
        "cant_cump_pi",
        "cant_vz_pi",
        "cant_cump_ech",
        "cant_vz_ech",
    ],
    axis=1,
    inplace=True,
)

tbl_rap_ov.head(12)

==========> 3. Export Report's

In [14]:
tbl_data_set.to_csv(
    r"PATH_TO_OUTPUT/FILE_NAME.csv",
    sep="|",
    decimal=".",
    mode="w",
    index=False,
)
tbl_ibd_statistic_m.to_csv(
    r"PATH_TO_OUTPUT/FILE_NAME.csv",
    sep="|",
    decimal=".",
    mode="w",
    index=False,
)
tbl_rap_ov.to_csv(
    r"PATH_TO_OUTPUT/FILE_NAME.csv",
    sep="|",
    decimal=".",
    mode="w",
    index=False,
)
graph_abat.write_image(
    r"PATH_TO_OUTPUT/GRAPH_NAME.jpeg"
)
graph_prog_vs_real.write_image(
    r"PATH_TO_OUTPUT/GRAPH_NAME.jpeg"
)